# Ejercicio 4: Modelo Probabilístico

## Objetivo de la práctica
- Comprender los componentes del modelo vectorial mediante cálculos manuales y observación directa.
- Aplicar el modelo de espacio vectorial con TF-IDF para recuperar documentos relevantes.
- Comparar la recuperación con BM25 frente a TF-IDF.
- Analizar visualmente las diferencias entre los modelos.
- Evaluar si los rankings generados son consistentes con lo que considerarías documentos relevantes.

## Parte 0: Carga del Corpus

In [12]:
from sklearn.datasets import fetch_20newsgroups

newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
newsgroupsdocs = newsgroups.data

In [27]:
#tipo de newsgroupsdocs
print(type(newsgroupsdocs))

<class 'list'>


In [46]:
#ver un documento
print(newsgroupsdocs[0])



I am sure some bashers of Pens fans are pretty confused about the lack
of any kind of posts about the recent Pens massacre of the Devils. Actually,
I am  bit puzzled too and a bit relieved. However, I am going to put an end
to non-PIttsburghers' relief with a bit of praise for the Pens. Man, they
are killing those Devils worse than I thought. Jagr just showed you why
he is much better than his regular season stats. He is also a lot
fo fun to watch in the playoffs. Bowman should let JAgr have a lot of
fun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was very disappointed not to see the Islanders lose the final
regular season game.          PENS RULE!!!




In [30]:
#peso de newsgroupsdocs
print(newsgroupsdocs.__sizeof__())

150808


In [ ]:
#vectorizando newsgroupsdocs 
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
#tamaño del vector
X = vectorizer.fit_transform(newsgroupsdocs)
print(X.shape)
corpus_vect=vectorizer.transform(newsgroupsdocs)
corpus_vect.toarray()
#tamaño del vector
print(corpus_vect.shape)


(18846, 134410)
(18846, 134410)


In [49]:
# query chicken
query = ["some"]
#vectorizando query
query_vect = vectorizer.transform(query)
#valor de la query
print(query_vect.toarray())
#similaridad entre query y corpus
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(query_vect, corpus_vect)
#similaridad entre query y corpus
print(similarity)

[[0. 0. 0. ... 0. 0. 0.]]
[[0.03797469 0.         0.02884581 ... 0.         0.         0.08310875]]


## Parte 1: Cálculo de TF, DF, IDF y TF-IDF

### Actividad 
1. Utiliza el corpus cargado.
2. Construye la matriz de términos (TF), y calcula la frecuencia de documentos (DF)
3. Calcula TF-IDF utilizando sklearn.
4. Visualiza los valores en un DataFrame para analizar las diferencias entre los términos.

In [13]:
import pandas as pd
import re

In [14]:
# Paso 1: Construcción del vocabulario (limpieza básica de texto)
vocabulary = set()
for doc in newsgroups:
    # Eliminamos caracteres no alfabéticos y convertimos a minúsculas
    words = re.findall(r'\b\w+\b', doc.lower())
    vocabulary.update(words)

vocabulary = list(vocabulary)  # Convertimos el set en lista para mantener el orden

# Paso 2: Construcción de la matriz término-documento (frecuencia de término)
dict_tf = {}
for i, doc in enumerate(newsgroupsdocs):
    dict_tfd = {}
    words = re.findall(r'\b\w+\b', doc.lower())  # Limpiamos igual que antes
    for term in vocabulary:
        dict_tfd[term] = words.count(term)
    dict_tf[i] = dict_tfd

# Paso 3: Crear la matriz término-documento como DataFrame
matriz = pd.DataFrame.from_dict(dict_tf, orient='index')  # Documentos como filas
print(matriz)


       data  descr  filenames  target_names  target
0         0      0          0             0       0
1         0      0          0             0       0
2         0      0          0             0       0
3         6      0          0             0       0
4         0      0          0             0       0
...     ...    ...        ...           ...     ...
18841     0      0          0             0       0
18842     0      0          0             0       0
18843     0      0          0             0       0
18844     0      0          0             0       0
18845     0      0          0             0       0

[18846 rows x 5 columns]


In [16]:
import numpy as np

In [18]:
# Paso 4: Calcular el IDF para cada término
N = len(newsgroupsdocs)
df = (matriz > 0).sum(axis=0)  # Número de documentos en los que aparece cada término
idf = np.log((N) / (df + 1))  # Suma 1 para evitar división por cero

# Paso 5: Calcular la matriz TF-IDF
matriz_tfidf = matriz * idf

# Mostrar matrices
print("\nValores IDF:")
print(idf.head(20))  # Mostrar los primeros 20 términos

print("\nMatriz TF-IDF:")
print(matriz_tfidf.iloc[:, :20])  # Mostrar primeras 20 columnas


Valores IDF:
data            3.192484
descr           9.844056
filenames       7.764614
target_names    9.844056
target          5.056564
dtype: float64

Matriz TF-IDF:
            data  descr  filenames  target_names  target
0       0.000000    0.0        0.0           0.0     0.0
1       0.000000    0.0        0.0           0.0     0.0
2       0.000000    0.0        0.0           0.0     0.0
3      19.154905    0.0        0.0           0.0     0.0
4       0.000000    0.0        0.0           0.0     0.0
...          ...    ...        ...           ...     ...
18841   0.000000    0.0        0.0           0.0     0.0
18842   0.000000    0.0        0.0           0.0     0.0
18843   0.000000    0.0        0.0           0.0     0.0
18844   0.000000    0.0        0.0           0.0     0.0
18845   0.000000    0.0        0.0           0.0     0.0

[18846 rows x 5 columns]


In [26]:
# Seleccionar los 20 términos más frecuentes (con menor IDF)
top_terms = idf.sort_values().head().index  # Los términos más comunes

# Crear un DataFrame solo con estos términos para todos los documentos
visual_df = matriz_tfidf[top_terms]

# Mostrar la tabla
print("\nVisualización de TF-IDF:")
print(visual_df)



Visualización de TF-IDF:
            data  target  filenames  descr  target_names
0       0.000000     0.0        0.0    0.0           0.0
1       0.000000     0.0        0.0    0.0           0.0
2       0.000000     0.0        0.0    0.0           0.0
3      19.154905     0.0        0.0    0.0           0.0
4       0.000000     0.0        0.0    0.0           0.0
...          ...     ...        ...    ...           ...
18841   0.000000     0.0        0.0    0.0           0.0
18842   0.000000     0.0        0.0    0.0           0.0
18843   0.000000     0.0        0.0    0.0           0.0
18844   0.000000     0.0        0.0    0.0           0.0
18845   0.000000     0.0        0.0    0.0           0.0

[18846 rows x 5 columns]


## Parte 2: Ranking de documentos usando TF-IDF

### Actividad 

1. Dada una consulta, construye el vector de consulta
2. Calcula la similitud coseno entre la consulta y cada documento usando los vectores TF-IDF
3. Genera un ranking de los documentos ordenados por relevancia.
4. Muestra los resultados en una tabla.

## Parte 3: Ranking con BM25

### Actividad 

1. Implementa un sistema de recuperación usando el modelo BM25.
2. Usa la misma consulta del ejercicio anterior.
3. Calcula el score BM25 para cada documento y genera un ranking.
4. Compara manualmente con el ranking de TF-IDF.

## Parte 4: Comparación visual entre TF-IDF y BM25

### Actividad 

1. Utiliza un gráfico de barras para visualizar los scores obtenidos por cada documento según TF-IDF y BM25.
2. Compara los rankings visualmente.
3. Identifica: ¿Qué documentos obtienen scores más altos en un modelo que en otro?
4. Sugiere: ¿A qué se podría deber esta diferencia?

## Parte 5: Evaluación con consulta relevante

### Actividad 

1. Elige una consulta y define qué documentos del corpus deberían considerarse relevantes.
2. Evalúa Precision@3 o MAP para los rankings generados con TF-IDF y BM25.
3. Responde: ¿Cuál modelo da mejores resultados respecto a tu criterio de relevancia?